# real-time call transcribe with es-US on Amazon Connect 8kHZ wav files



### *确定你本机配置了aksk
### *从S3上下载了录音文件到本地目录
aws s3 sync s3://xxx/ .


In [ ]:
# step3. Generate hypothesis.txt by looping real-time calling Amazon Transcribe
!pip install aiofile amazon-transcribe 

In [ ]:
# remove punctuation
def norm( line ):
    punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~“”？，！【】（）、。：；’‘……￥·"""
    dicts={i:'' for i in punctuation}
    punc_table=str.maketrans(dicts)
    line=line.translate(punc_table)
    return line

In [ ]:
%%time
#Amazon Transcribe Streaming SDK https://pypi.org/project/amazon-transcribe/
import os
import asyncio
# This example uses aiofile for asynchronous file reads.
# It's not a dependency of the project but can be installed
# with `pip install aiofile`.
import aiofile
import nest_asyncio
from datetime import datetime

from amazon_transcribe.client import TranscribeStreamingClient
from amazon_transcribe.handlers import TranscriptResultStreamHandler
from amazon_transcribe.model import TranscriptEvent

nest_asyncio.apply()
globalresult = ""
"""
Here's an example of a custom event handler you can extend to
process the returned transcription results as needed. This
handler will simply print the text out to your interpreter.
"""
class MyEventHandler(TranscriptResultStreamHandler):
    
    async def handle_transcript_event(self, transcript_event: TranscriptEvent):
        # This handler can be implemented to handle transcriptions as needed.
        # Here's an example to get started.
        global globalresult
        results = transcript_event.transcript.results
        
        for result in results:
            if( not result.is_partial):
                for alt in result.alternatives:
                    globalresult = globalresult +" "+ alt.transcript
                    #print(globalresult)

async def basic_transcribe(audiofile):
    # Setup up our client with our chosen AWS region
    client = TranscribeStreamingClient(region="us-east-1")

    # Start transcription to generate our async stream
    stream = await client.start_stream_transcription(
        language_code="es-US",
        media_sample_rate_hz=8000,
        media_encoding="pcm",
    )

    async def write_chunks():
        # An example file can be found at tests/integration/assets/test.wav
        # NOTE: For pre-recorded files longer than 5 minutes, the sent audio
        # chunks should be rate limited to match the realtime bitrate of the
        # audio stream to avoid signing issues.
        async with aiofile.AIOFile(audiofile, 'rb') as afp:
            reader = aiofile.Reader(afp, chunk_size=1024 * 8)
            async for chunk in reader:
                await stream.input_stream.send_audio_event(audio_chunk=chunk)
        await stream.input_stream.end_stream()

    # Instantiate our handler and start processing events
    handler = MyEventHandler(stream.output_stream)
    await asyncio.gather(write_chunks(), handler.handle_events())

In [ ]:
import os

path ="./wav-es-US"
#we shall store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))

#print all the file names
#for name in filelist:
#    print(name)

In [ ]:
#hypothesis
loop = asyncio.get_event_loop()
#wav_files = ['file.wav']
index = 0
print("---start-----")
for audiofile in filelist:
    if not audiofile.endswith('.wav'):
        print(f"{audiofile} is NOT proper file format")
        continue
    print("Audio:  {0}".format(audiofile))
    #audiofile = './wav-es-US/'+audiofile
    t1 = datetime.now()
    loop.run_until_complete(basic_transcribe(audiofile))
    t2 = datetime.now()
    t = t2-t1
    #print("Latency:  {0}".format(t))
    #print("asdfasdf"+norm(globalresult))
    #print(f"file name:{audiofile} , result: {norm(globalresult)}")
    print(f"file name:{audiofile} , result: {globalresult}")
    #hypothesis.at[index,'LATENCY']=t
    index = index+1

print("---end-----")
#loop.stop()    